# OSM Bronze → Silver: Energy & Infrastructure Features

Extracts OpenStreetMap features for Spain focused on **energy, extraction, transport infrastructure** (plus corridor and constraint layers) into a single silver Parquet table.

## Tooling choice: **Pyosmium**

| Tool | Pros | Cons | Verdict |
|------|------|------|--------|
| **Pyosmium** | Native Python, streaming, built-in geometry (WKBFactory), area callback for multipolygons, single-process | Requires ~2–4 GB RAM for Spain PBF | ✅ **Selected** |
| osmium-tool | Fast CLI filter | Outputs PBF, needs separate parser; no direct Parquet | Extra step |
| GDAL/ogr2ogr | OSM driver, GeoPackage | Slow for 1.3 GB; not recommended for large extracts | ❌ |
| osm2pgsql | Battle-tested, PostGIS | Outputs Postgres, not Parquet; needs export | ❌ |
| Spark+Sedona | Scalable | Overkill for single country; infra complexity | ❌ |

**Pyosmium** provides: (1) single-pass PBF read with `with_locations()`, (2) `area` callback for relations → automatic multipolygon assembly, (3) `WKBFactory` for Point/LineString/MultiPolygon, (4) batch-friendly accumulation + Parquet writes.

## Input / Output

| Layer | S3 path |
|-------|--------|
| Bronze | `s3://<bucket>/bronze/osm/spain-latest.osm.pbf` |
| Silver | `s3://<bucket>/silver/osm/features_energy/country=ES/snapshot_date=YYYY-MM-DD/feature_type=<TYPE>/part-*.parquet` |

## Feature types (31 total)

Energy/infra + Water & wetness + Protected + Human footprint + Landuse + Barriers + Green/urban + Trails

In [1]:
# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────

COUNTRY = "ES"
S3_BUCKET = "ie-datalake"
BRONZE_PREFIX = "bronze/osm"
SILVER_PREFIX = "silver/osm/features_energy"
AWS_PROFILE = "486717354268_PowerUserAccess"

# Input PBF (local path – already downloaded)
PBF_PATH = "../data/tmpptqi3rfv.osm.pbf"

# Bbox split: Spain ~(-9.5,36) to (4.5,43.5), 20 tiles = 4x5 grid
N_BBOX_COLS = 4
N_BBOX_ROWS = 5
SPAIN_BBOX = (-9.5, 36.0, 4.5, 43.5)  # minlon, minlat, maxlon, maxlat
BBOX_DIR = "../data/osm_bboxes"  # output of split, input for process loop

# Snapshot date
from datetime import date
SNAPSHOT_DATE = date.today().isoformat()

# Batch size for Parquet writes
BATCH_SIZE = 100_000
PARQUET_COMPRESSION = "snappy"

# Write: direct to S3 per batch (no parallel, no local-first)
STORAGE_OPTIONS = {"profile": AWS_PROFILE}

In [5]:
%pip install -q osmium geopandas pyarrow s3fs h3

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations

import json
import logging
import os
import shutil
import subprocess
import tempfile
from datetime import date
from pathlib import Path
from typing import Any

import geopandas as gpd
import h3
import osmium as osm
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import s3fs
from shapely import wkb
from shapely.geometry import shape

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S",
    force=True,
)
log = logging.getLogger("osm_silver")

## Feature type detection & tag filters

Each feature type has tag conditions (OR within group). First match wins. Promote typed columns per spec.

In [3]:
FEATURE_TYPES = [
    # Energy & infrastructure (original)
    "PIPELINES", "POWER_LINES", "POWER_SUBSTATIONS", "POWER_PLANTS",
    "INDUSTRIAL_AREAS", "STORAGE_TANKS", "FUEL_STATIONS",
    "PORTS_TERMINALS", "AIRPORTS", "ROADS", "RAIL",
    "ADMIN_BOUNDARIES", "PROTECTED_AREAS",
    # A) Water & wetness
    "WATERWAYS", "WATERBODIES", "WETLANDS", "COASTLINE",
    "WATER_BARRIERS", "WATER_INFRA_POI",
    # B) Protected / regulated
    "RESTRICTED_AREAS",
    # C) Human footprint
    "BUILDINGS", "AMENITIES_POI", "WASTE_POLLUTION",
    # D) Landuse / habitat
    "LANDUSE_AGRICULTURE", "FORESTRY_MANAGED", "NATURAL_HABITATS",
    # E) Barriers & fragmentation
    "BARRIERS", "LINEAR_DISTURBANCE",
    # F) Green/urban ecology
    "PARKS_GREEN_URBAN", "TREE_ROWS_HEDGEROWS",
    # G) Soft transport
    "TRAILS_TRACKS",
]


def _tags_dict(obj) -> dict[str, str]:
    return {t.k: t.v for t in obj.tags}


def _get_tag(tags: dict[str, str], *keys: str) -> str | None:
    for k in keys:
        if k in tags and tags[k]:
            return tags[k]
    return None


def match_feature_type(tags: dict[str, str]) -> tuple[str | None, dict[str, Any]]:
    """
    Returns (feature_type, promoted_cols).
    First matching feature type wins. promoted_cols holds typed columns for that type.
    """
    # A) PIPELINES
    if tags.get("man_made") == "pipeline" or tags.get("pipeline") or tags.get("substance") or tags.get("location") in ("underground", "overground"):
        return "PIPELINES", {
            "pipeline": tags.get("pipeline"),
            "substance": tags.get("substance"),
            "location": tags.get("location"),
            "operator": tags.get("operator"),
            "ref": tags.get("ref"),
        }

    # B) POWER_LINES
    if tags.get("power") in ("line", "minor_line", "cable"):
        return "POWER_LINES", {
            "power": tags.get("power"),
            "voltage": tags.get("voltage"),
            "cables": tags.get("cables"),
            "location": tags.get("location"),
            "operator": tags.get("operator"),
        }

    # C) POWER_SUBSTATIONS
    if tags.get("power") == "substation":
        return "POWER_SUBSTATIONS", {
            "substation": tags.get("substation"),
            "voltage": tags.get("voltage"),
            "operator": tags.get("operator"),
        }

    # D) POWER_PLANTS
    if tags.get("power") in ("plant", "generator"):
        return "POWER_PLANTS", {
            "plant_source": tags.get("plant:source") or tags.get("plant_source"),
            "generator_source": tags.get("generator:source") or tags.get("generator_source"),
            "operator": tags.get("operator"),
            "capacity": tags.get("plant:output:electricity") or tags.get("capacity"),
            "power": tags.get("power"),
        }

    # E) INDUSTRIAL_AREAS
    if tags.get("landuse") == "industrial" or tags.get("industrial") or tags.get("man_made") == "works":
        return "INDUSTRIAL_AREAS", {
            "landuse": tags.get("landuse"),
            "industrial": tags.get("industrial"),
            "man_made": tags.get("man_made"),
            "operator": tags.get("operator"),
        }

    # F) STORAGE_TANKS
    if tags.get("man_made") == "storage_tank" or tags.get("landuse") == "depot" or tags.get("industrial") in ("oil", "chemical"):
        return "STORAGE_TANKS", {
            "man_made": tags.get("man_made"),
            "content": tags.get("content") or tags.get("substance"),
            "operator": tags.get("operator"),
        }

    # G) FUEL_STATIONS
    if tags.get("amenity") == "fuel":
        return "FUEL_STATIONS", {
            "amenity": tags.get("amenity"),
            "brand": tags.get("brand"),
            "operator": tags.get("operator"),
            "opening_hours": tags.get("opening_hours"),
        }

    # H) PORTS_TERMINALS
    if tags.get("harbour") or tags.get("landuse") == "port" or tags.get("man_made") == "pier" or tags.get("amenity") == "ferry_terminal":
        return "PORTS_TERMINALS", {
            "harbour": tags.get("harbour"),
            "landuse": tags.get("landuse"),
            "man_made": tags.get("man_made"),
            "operator": tags.get("operator"),
        }

    # I) AIRPORTS
    if tags.get("aeroway") in ("aerodrome", "terminal", "runway", "taxiway"):
        return "AIRPORTS", {
            "aeroway": tags.get("aeroway"),
            "iata": tags.get("iata"),
            "icao": tags.get("icao"),
            "operator": tags.get("operator"),
        }

    # G) TRAILS_TRACKS (before ROADS – more specific)
    if tags.get("highway") in ("path", "track", "footway", "bridleway", "cycleway"):
        return "TRAILS_TRACKS", {
            "highway": tags.get("highway"),
            "surface": tags.get("surface"),
            "access": tags.get("access"),
            "sac_scale": tags.get("sac_scale"),
            "name": tags.get("name"),
        }

    # J) ROADS
    if tags.get("highway"):
        return "ROADS", {
            "highway": tags.get("highway"),
            "surface": tags.get("surface"),
            "bridge": tags.get("bridge"),
            "tunnel": tags.get("tunnel"),
            "oneway": tags.get("oneway"),
            "maxspeed": tags.get("maxspeed"),
            "access": tags.get("access"),
            "ref": tags.get("ref"),
        }

    # K) RAIL
    if tags.get("railway") in ("rail", "light_rail", "subway", "tram"):
        return "RAIL", {
            "railway": tags.get("railway"),
            "electrified": tags.get("electrified"),
            "operator": tags.get("operator"),
            "usage": tags.get("usage"),
        }

    # L) ADMIN_BOUNDARIES
    if tags.get("boundary") == "administrative":
        return "ADMIN_BOUNDARIES", {
            "admin_level": tags.get("admin_level"),
            "boundary": tags.get("boundary"),
            "name": tags.get("name"),
        }

    # M) PROTECTED_AREAS
    if tags.get("boundary") == "protected_area" or tags.get("leisure") == "nature_reserve":
        return "PROTECTED_AREAS", {
            "protect_class": tags.get("protect_class"),
            "designation": tags.get("designation"),
            "operator": tags.get("operator"),
            "name": tags.get("name"),
        }

    # A) Water & wetness
    if tags.get("waterway") in ("dam", "weir", "lock_gate") or tags.get("man_made") == "dam":
        return "WATER_BARRIERS", {
            "man_made": tags.get("man_made"),
            "waterway": tags.get("waterway"),
            "operator": tags.get("operator"),
            "name": tags.get("name"),
        }
    if tags.get("waterway"):
        return "WATERWAYS", {
            "waterway": tags.get("waterway"),
            "name": tags.get("name"),
            "intermittent": tags.get("intermittent"),
            "tunnel": tags.get("tunnel"),
            "width": tags.get("width"),
        }
    if tags.get("natural") == "water" or tags.get("water"):
        return "WATERBODIES", {
            "water": tags.get("water"),
            "name": tags.get("name"),
            "reservoir": tags.get("reservoir"),
        }
    if tags.get("natural") == "wetland" or tags.get("wetland"):
        return "WETLANDS", {"wetland": tags.get("wetland"), "name": tags.get("name")}
    if tags.get("natural") == "coastline":
        return "COASTLINE", {"name": tags.get("name")}
    if tags.get("man_made") in ("water_tower",) or tags.get("amenity") == "drinking_water" or tags.get("emergency") == "water_tank":
        return "WATER_INFRA_POI", {
            "man_made": tags.get("man_made"),
            "amenity": tags.get("amenity"),
            "name": tags.get("name"),
            "operator": tags.get("operator"),
        }

    # B) RESTRICTED_AREAS
    if tags.get("military") or tags.get("landuse") == "military" or tags.get("boundary") == "military":
        return "RESTRICTED_AREAS", {
            "military": tags.get("military"),
            "landuse": tags.get("landuse"),
            "name": tags.get("name"),
        }

    # C) Human footprint
    if tags.get("building"):
        return "BUILDINGS", {
            "building": tags.get("building"),
            "building_levels": tags.get("building:levels"),
            "name": tags.get("name"),
        }
    if tags.get("amenity") in ("school", "hospital", "university", "marketplace", "prison", "waste_disposal", "recycling"):
        return "AMENITIES_POI", {
            "amenity": tags.get("amenity"),
            "name": tags.get("name"),
            "operator": tags.get("operator"),
        }
    if tags.get("landuse") == "landfill" or tags.get("man_made") == "wastewater_plant":
        return "WASTE_POLLUTION", {
            "landuse": tags.get("landuse"),
            "man_made": tags.get("man_made"),
            "name": tags.get("name"),
            "operator": tags.get("operator"),
        }

    # D) Landuse / habitat
    if tags.get("landuse") in ("farmland", "farmyard", "orchard", "vineyard", "meadow"):
        return "LANDUSE_AGRICULTURE", {
            "landuse": tags.get("landuse"),
            "crop": tags.get("crop"),
            "name": tags.get("name"),
        }
    if tags.get("landuse") == "forest" or tags.get("forest"):
        return "FORESTRY_MANAGED", {
            "landuse": tags.get("landuse"),
            "forest": tags.get("forest"),
            "name": tags.get("name"),
        }
    if tags.get("natural") in ("wood", "heath", "scrub", "grassland", "bare_rock", "sand", "beach", "cliff"):
        return "NATURAL_HABITATS", {"natural": tags.get("natural"), "name": tags.get("name")}

    # E) Barriers & fragmentation
    if tags.get("natural") == "tree_row" or tags.get("barrier") == "hedge":
        return "TREE_ROWS_HEDGEROWS", {"natural": tags.get("natural"), "barrier": tags.get("barrier")}
    if tags.get("barrier") in ("fence", "wall", "hedge", "gate", "bollard"):
        return "BARRIERS", {"barrier": tags.get("barrier"), "access": tags.get("access")}
    if tags.get("man_made") in ("cutline", "embankment") or tags.get("barrier") == "ditch":
        return "LINEAR_DISTURBANCE", {"man_made": tags.get("man_made"), "barrier": tags.get("barrier")}

    # F) Green/urban ecology
    if tags.get("leisure") in ("park", "garden", "common", "golf_course", "pitch") or tags.get("landuse") == "recreation_ground":
        return "PARKS_GREEN_URBAN", {
            "leisure": tags.get("leisure"),
            "landuse": tags.get("landuse"),
            "name": tags.get("name"),
        }

    return None, {}

In [4]:
# All promoted columns – every row gets all keys (str or None) for consistent Parquet schema
PROMOTED_KEYS = (
    "pipeline", "substance", "location", "operator", "ref",
    "power", "voltage", "cables", "substation", "plant_source", "generator_source", "capacity",
    "landuse", "industrial", "man_made", "content", "amenity", "brand", "opening_hours",
    "harbour", "aeroway", "iata", "icao",
    "highway", "surface", "bridge", "tunnel", "oneway", "maxspeed", "access",
    "railway", "electrified", "usage",
    "admin_level", "boundary", "protect_class", "designation",
    "waterway", "water", "wetland", "intermittent", "width", "reservoir",
    "military", "building", "building_levels", "crop", "forest",
    "natural", "barrier", "sac_scale", "leisure",
)

# Keys to keep in tags_json (prune to avoid huge blobs)
TAGS_JSON_KEYS = frozenset([
    "name", "operator", "ref", "brand", "opening_hours",
    "pipeline", "substance", "location", "power", "voltage", "cables",
    "substation", "plant:source", "generator:source", "plant:output:electricity", "capacity",
    "landuse", "industrial", "man_made", "content", "amenity",
    "harbour", "aeroway", "iata", "icao",
    "highway", "surface", "bridge", "tunnel", "oneway", "maxspeed", "access",
    "railway", "electrified", "usage",
    "boundary", "admin_level", "protect_class", "designation", "leisure",
    "waterway", "water", "wetland", "intermittent", "width", "reservoir",
    "military", "building", "building:levels", "crop", "forest",
    "natural", "barrier", "sac_scale", "emergency",
])


def prune_tags(tags: dict[str, str]) -> dict[str, str]:
    return {k: v for k, v in tags.items() if k in TAGS_JSON_KEYS and not k.startswith("name:")}

## Pyosmium handler: extract features → batch Parquet

In [5]:
def add_h3_from_point(lat: float, lon: float) -> dict[str, str]:
    try:
        h9 = h3.latlng_to_cell(lat, lon, 9)
        return {
            "h3_6": h3.cell_to_parent(h9, 6),
            "h3_7": h3.cell_to_parent(h9, 7),
            "h3_8": h3.cell_to_parent(h9, 8),
            "h3_9": h9,
        }
    except Exception:
        return {"h3_6": None, "h3_7": None, "h3_8": None, "h3_9": None}


def centroid_from_geom(geom) -> tuple[float | None, float | None]:
    """Use representative_point for polygons (PointOnSurface), centroid for points/lines."""
    if geom is None or geom.is_empty:
        return None, None
    try:
        if geom.geom_type in ("Polygon", "MultiPolygon"):
            pt = geom.representative_point()
        else:
            pt = geom.centroid
        return float(pt.x), float(pt.y)
    except Exception:
        return None, None


def length_area_from_geom(geom) -> tuple[float, float]:
    """Length (m) and area (m²) in EPSG:25830 for Gold layer."""
    if geom is None or geom.is_empty:
        return 0.0, 0.0
    try:
        from shapely.ops import transform
        from pyproj import Transformer
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:25830", always_xy=True)
        geom_p = transform(lambda x, y: transformer.transform(x, y), geom)
        length = geom_p.length if geom_p.geom_type in ("LineString", "MultiLineString") else 0.0
        area = geom_p.area if geom_p.geom_type in ("Polygon", "MultiPolygon") else 0.0
        return float(length), float(area)
    except Exception:
        return 0.0, 0.0

In [6]:
class EnergyFeaturesHandler(osm.SimpleHandler):
    """
    Extracts OSM features matching energy/infrastructure tags.
    - Nodes → Point
    - Ways (non-closed) → LineString
    - Areas (closed ways + multipolygon relations) → Polygon/MultiPolygon
    """

    def __init__(self, wkb_factory: osm.geom.WKBFactory, country: str, snapshot_date: str, batch_callback):
        super().__init__()
        self.wkb_factory = wkb_factory
        self.country = country
        self.snapshot_date = snapshot_date
        self.batch_callback = batch_callback
        self._batch: list[dict] = []
        self._counts: dict[str, int] = {ft: 0 for ft in FEATURE_TYPES}

    def _emit(self, osm_id: str, osm_type: str, feature_type: str, geom_wkb: bytes, tags: dict, promoted: dict):
        if geom_wkb is None:
            geom = None
        elif isinstance(geom_wkb, bytes):
            geom = wkb.loads(geom_wkb)
        else:
            geom = wkb.loads(bytes.fromhex(geom_wkb))  # Pyosmium returns hex string
        lon, lat = centroid_from_geom(geom)
        if lat is None or lon is None:
            return
        length_m, area_m2 = length_area_from_geom(geom)
        h3_cols = add_h3_from_point(lat, lon)
        tags_pruned = prune_tags(tags)
        # All promoted columns as str/None – consistent schema across partitions for Athena
        promoted_full = {k: (str(v) if v is not None else None) for k in PROMOTED_KEYS for v in [promoted.get(k)]}
        row = {
            "osm_id": osm_id,
            "osm_type": osm_type,
            "feature_type": feature_type,
            "geometry": geom,
            "length_m": length_m,
            "area_m2": area_m2,
            "centroid_lon": lon,
            "centroid_lat": lat,
            **h3_cols,
            "name": tags.get("name"),
            "tags_json": json.dumps(tags_pruned) if tags_pruned else None,
            "snapshot_date": self.snapshot_date,
            "country": self.country,
            **promoted_full,
        }
        self._batch.append(row)
        self._counts[feature_type] = self._counts.get(feature_type, 0) + 1
        if len(self._batch) >= BATCH_SIZE:
            self._flush()

    def _flush(self):
        if not self._batch:
            return
        self.batch_callback(self._batch)
        self._batch = []

    def node(self, n: osm.Node):
        if not n.location.valid():
            return
        tags = _tags_dict(n)
        ft, promoted = match_feature_type(tags)
        if ft is None:
            return
        try:
            wkb_bytes = self.wkb_factory.create_point(n)
        except Exception:
            return
        self._emit(f"n{n.id}", "node", ft, wkb_bytes, tags, promoted)

    def way(self, w: osm.Way):
        if w.is_closed() and len(w.nodes) >= 4:
            return  # Let area callback handle closed ways
        tags = _tags_dict(w)
        ft, promoted = match_feature_type(tags)
        if ft is None:
            return
        try:
            wkb_bytes = self.wkb_factory.create_linestring(w)
        except Exception:
            return
        self._emit(f"w{w.id}", "way", ft, wkb_bytes, tags, promoted)

    def area(self, a: osm.Area):
        tags = _tags_dict(a)
        ft, promoted = match_feature_type(tags)
        if ft is None:
            return
        try:
            wkb_bytes = self.wkb_factory.create_multipolygon(a)
        except Exception:
            return
        osm_type = "way" if a.from_way() else "relation"
        osm_id = f"w{a.orig_id()}" if a.from_way() else f"r{a.orig_id()}"
        self._emit(osm_id, osm_type, ft, wkb_bytes, tags, promoted)

In [7]:
def build_schema_with_promoted() -> dict[str, pa.DataType]:
    base = {
        "osm_id": pa.string(),
        "osm_type": pa.string(),
        "feature_type": pa.string(),
        "centroid_lon": pa.float64(),
        "centroid_lat": pa.float64(),
        "h3_6": pa.string(),
        "h3_7": pa.string(),
        "h3_8": pa.string(),
        "h3_9": pa.string(),
        "name": pa.string(),
        "tags_json": pa.string(),
        "snapshot_date": pa.string(),
        "country": pa.string(),
    }
    for k in PROMOTED_KEYS:
        base[k] = pa.string()
    return base


def _table_no_dictionary(tbl: pa.Table) -> pa.Table:
    """Decode all dictionary columns to plain string – avoids Parquet merge errors."""
    for i in range(tbl.num_columns):
        if pa.types.is_dictionary(tbl.schema.field(i).type):
            tbl = tbl.set_column(i, tbl.schema.field(i).name, tbl.column(i).dictionary_decode())
    return tbl

In [8]:
def get_part_counter_from_s3(
    s3_fs: s3fs.S3FileSystem,
    country: str,
    snapshot_date: str,
) -> dict[str, int]:
    """Load part_counter from existing S3 files (for resume)."""
    base = f"{S3_BUCKET}/{SILVER_PREFIX}/country={country}/snapshot_date={snapshot_date}"
    part_counter = {ft: 0 for ft in FEATURE_TYPES}
    for ft in FEATURE_TYPES:
        prefix = f"{base}/feature_type={ft}/"
        files = s3_fs.glob(f"{prefix}part-*.parquet")
        if files:
            nums = sorted(int(f.split("part-")[1].split(".")[0]) for f in files)
            part_counter[ft] = max(nums) if nums else 0
    return part_counter


def process_pbf_and_write(
    pbf_path: str,
    country: str,
    snapshot_date: str,
    s3_fs: s3fs.S3FileSystem,
    part_counter: dict[str, int] | None = None,
) -> dict[str, int]:
    """Process PBF, write directly to S3 per batch. part_counter persists across bbox runs."""
    wkb_factory = osm.geom.WKBFactory()
    if part_counter is None:
        part_counter = {ft: 0 for ft in FEATURE_TYPES}
    s3_prefix = f"{S3_BUCKET}/{SILVER_PREFIX}/country={country}/snapshot_date={snapshot_date}"

    STRING_COLS = list(PROMOTED_KEYS) + ["osm_id", "osm_type", "feature_type", "name", "tags_json", "snapshot_date", "country", "h3_6", "h3_7", "h3_8", "h3_9"]

    def _coerce_string_cols(df):
        """Ensure all string columns are str|None (no NaN) for consistent Parquet schema."""
        for c in STRING_COLS:
            if c not in df.columns:
                continue
            s = df[c]
            null_mask = s.isna()
            df[c] = s.where(~null_mask).astype(str).mask(null_mask, None)
        return df

    from geopandas.io.arrow import _geopandas_to_arrow

    def on_batch(batch: list[dict]):
        gdf = gpd.GeoDataFrame(batch, crs="EPSG:4326", geometry="geometry")
        for ft, sub in gdf.groupby("feature_type"):
            part_counter[ft] = part_counter.get(ft, 0) + 1
            out_path = f"s3://{s3_prefix}/feature_type={ft}/part-{part_counter[ft]:05d}.parquet"
            sub = _coerce_string_cols(sub.copy())
            try:
                tbl = _geopandas_to_arrow(sub, index=False, geometry_encoding="WKB")
                tbl = _table_no_dictionary(tbl)
                pq.write_table(
                    tbl,
                    out_path,
                    compression=PARQUET_COMPRESSION,
                    filesystem=s3_fs,
                    use_dictionary=False,
                    row_group_size=len(tbl),
                )
            except Exception as e:
                log.error("Write failed %s: %s", out_path, e)
                raise RuntimeError(f"Failed to write {out_path}: {e}") from e
            log.info("  Wrote %s: %d rows", out_path, len(sub))

    handler = EnergyFeaturesHandler(wkb_factory, country, snapshot_date, on_batch)
    log.info("Applying handler to %s (with locations + areas)…", pbf_path)
    handler.apply_file(pbf_path, locations=True)
    handler._flush()
    return handler._counts


In [9]:
def split_pbf_into_bboxes(
    pbf_path: str,
    bbox: tuple[float, float, float, float],
    n_cols: int,
    n_rows: int,
    out_dir: str | Path,
) -> list[tuple[int, str]]:
    """
    Split PBF into n_cols x n_rows bbox tiles using osmium extract.
    Saves to out_dir (e.g. data/osm_bboxes). Run once.
    Requires: osmium-tool (brew install osmium-tool)
    """
    minlon, minlat, maxlon, maxlat = bbox
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    lon_step = (maxlon - minlon) / n_cols
    lat_step = (maxlat - minlat) / n_rows
    results: list[tuple[int, str]] = []

    for row in range(n_rows):
        for col in range(n_cols):
            idx = row * n_cols + col
            left = minlon + col * lon_step
            bottom = minlat + row * lat_step
            right = minlon + (col + 1) * lon_step
            top = minlat + (row + 1) * lat_step
            bbox_str = f"{left},{bottom},{right},{top}"
            out_path = out_dir / f"bbox_{idx:02d}.osm.pbf"

            log.info("Extracting bbox %d/%d: %s", idx + 1, n_cols * n_rows, bbox_str)
            r = subprocess.run(
                ["osmium", "extract", "-b", bbox_str, str(pbf_path), "-o", str(out_path), "-f", "pbf"],
                capture_output=True,
                text=True,
            )
            if r.returncode != 0:
                raise RuntimeError(f"osmium extract failed: {r.stderr}")
            results.append((idx, str(out_path)))

    return results

## Step 1: Split PBF into bboxes (run once)

**Requires:** `osmium-tool` (e.g. `brew install osmium-tool` or `apt install osmium-tool`)

In [12]:
# Split PBF into 20 bbox tiles. Output: BBOX_DIR/bbox_00.osm.pbf, ...
# Skips if BBOX_DIR has bbox_*.osm.pbf files, unless FORCE_RESPLIT=True.
FORCE_RESPLIT = True  # Set True to delete existing bboxes and re-split (e.g. after schema change)
bbox_dir = Path(BBOX_DIR)
existing = list(bbox_dir.glob("bbox_*.osm.pbf"))
if existing and not FORCE_RESPLIT:
    log.info("Skipping split: %d bbox files already in %s", len(existing), bbox_dir)
else:
    if existing and FORCE_RESPLIT:
        for f in existing:
            f.unlink()
        log.info("Removed %d existing bbox files (FORCE_RESPLIT=True)", len(existing))
    tiles = split_pbf_into_bboxes(PBF_PATH, SPAIN_BBOX, N_BBOX_COLS, N_BBOX_ROWS, out_dir=bbox_dir)
    log.info("Split into %d tiles in %s", len(tiles), bbox_dir.resolve())

22:06:02 [INFO] Removed 11 existing bbox files (FORCE_RESPLIT=True)
22:06:02 [INFO] Extracting bbox 1/20: -9.5,36.0,-6.0,37.5
22:06:10 [INFO] Extracting bbox 2/20: -6.0,36.0,-2.5,37.5
22:06:18 [INFO] Extracting bbox 3/20: -2.5,36.0,1.0,37.5
22:06:25 [INFO] Extracting bbox 4/20: 1.0,36.0,4.5,37.5
22:06:29 [INFO] Extracting bbox 5/20: -9.5,37.5,-6.0,39.0
22:06:35 [INFO] Extracting bbox 6/20: -6.0,37.5,-2.5,39.0
22:06:41 [INFO] Extracting bbox 7/20: -2.5,37.5,1.0,39.0
22:06:49 [INFO] Extracting bbox 8/20: 1.0,37.5,4.5,39.0
22:06:54 [INFO] Extracting bbox 9/20: -9.5,39.0,-6.0,40.5
22:06:59 [INFO] Extracting bbox 10/20: -6.0,39.0,-2.5,40.5
22:07:06 [INFO] Extracting bbox 11/20: -2.5,39.0,1.0,40.5
22:07:13 [INFO] Extracting bbox 12/20: 1.0,39.0,4.5,40.5
22:07:19 [INFO] Extracting bbox 13/20: -9.5,40.5,-6.0,42.0
22:07:25 [INFO] Extracting bbox 14/20: -6.0,40.5,-2.5,42.0
22:07:32 [INFO] Extracting bbox 15/20: -2.5,40.5,1.0,42.0
22:07:40 [INFO] Extracting bbox 16/20: 1.0,40.5,4.5,42.0
22:07:48 

## Step 2: Process bboxes → write to S3 (resumable)

Reads from `BBOX_DIR`. On resume: loads part_counter from S3, processes only remaining bbox files, deletes each after success.

**Why multiple part files per feature_type?** Each batch (100k rows) is grouped by feature_type; each group gets one file. So `part-00001`, `part-00002`, … = one file per batch that had that feature type.

In [11]:
MAX_BBOX_PROCESSING = 3  # None = no limit; set to 1 for dry run

fs = s3fs.S3FileSystem(profile=AWS_PROFILE)
bbox_dir = Path(BBOX_DIR)

# List bbox files (sorted). Empty = all done or split not run yet.
bbox_files = sorted(bbox_dir.glob("bbox_*.osm.pbf"))
if not bbox_files:
    log.info("No bbox files in %s. Run Step 1 (split) first, or all done.", bbox_dir)
else:
    # Load part_counter from S3 (for resume – continue numbering from existing parts)
    part_counter = get_part_counter_from_s3(fs, COUNTRY, SNAPSHOT_DATE)
    total_counts: dict[str, int] = {ft: 0 for ft in FEATURE_TYPES}
    to_process = bbox_files[:MAX_BBOX_PROCESSING] if MAX_BBOX_PROCESSING is not None else bbox_files
    if len(to_process) < len(bbox_files):
        log.info("Limiting to %d bbox(es) (MAX_BBOX_PROCESSING=%s)", len(to_process), MAX_BBOX_PROCESSING)

    for i, bbox_path in enumerate(to_process):
        log.info("Processing bbox %d/%d: %s", i + 1, len(to_process), bbox_path.name)
        try:
            counts = process_pbf_and_write(
                str(bbox_path), COUNTRY, SNAPSHOT_DATE, fs, part_counter=part_counter
            )
            for ft, n in counts.items():
                total_counts[ft] = total_counts.get(ft, 0) + n
            bbox_path.unlink()  # delete after success → resume skips it
        except Exception as e:
            log.error("Failed on %s: %s. Re-run this cell to resume.", bbox_path.name, e)
            raise

    total = sum(total_counts.values())
    log.info("Extraction complete: %d total features", total)
    for ft, n in sorted(total_counts.items(), key=lambda x: -x[1]):
        if n > 0:
            log.info("  %s: %d", ft, n)

21:02:32 [INFO] Found credentials in shared credentials file: ~/.aws/credentials
21:02:35 [INFO] Limiting to 3 bbox(es) (MAX_BBOX_PROCESSING=3)
21:02:35 [INFO] Processing bbox 1/3: bbox_09.osm.pbf
21:02:35 [INFO] Applying handler to ../data/osm_bboxes/bbox_09.osm.pbf (with locations + areas)…
21:22:52 [INFO]   Wrote s3://ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=AIRPORTS/part-00045.parquet: 21 rows
21:22:53 [INFO]   Wrote s3://ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=AMENITIES_POI/part-00046.parquet: 2437 rows
21:22:53 [INFO]   Wrote s3://ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=BARRIERS/part-00046.parquet: 6893 rows
21:22:53 [INFO]   Wrote s3://ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=BUILDINGS/part-00046.parquet: 204 rows
21:22:53 [INFO]   Wrote s3://ie-datalake/silver/osm/features_energy/country=ES/

KeyboardInterrupt: 

## QA checks & sanity metrics

In [13]:
def run_qa_checks(country: str, snapshot_date: str, s3_fs: s3fs.S3FileSystem) -> dict:
    base = f"{S3_BUCKET}/{SILVER_PREFIX}/country={country}/snapshot_date={snapshot_date}"
    results = {"feature_type": [], "n_rows": [], "n_files": [], "geometry_types": [], "centroid_valid_pct": []}

    for ft in FEATURE_TYPES:
        prefix = f"{base}/feature_type={ft}/"
        files = s3_fs.glob(f"{prefix}*.parquet")
        if not files:
            continue
        dfs = []
        for f in files:
            with s3_fs.open(f, "rb") as fh:
                dfs.append(gpd.read_parquet(fh))
        gdf = pd.concat(dfs, ignore_index=True)
        n = len(gdf)
        geom_types = gdf.geometry.geom_type.value_counts().to_dict() if hasattr(gdf, "geometry") else {}
        centroid_ok = gdf["centroid_lon"].notna().sum() / n * 100 if n else 0
        results["feature_type"].append(ft)
        results["n_rows"].append(n)
        results["n_files"].append(len(files))
        results["geometry_types"].append(geom_types)
        results["centroid_valid_pct"].append(round(centroid_ok, 2))

    return pd.DataFrame(results)

In [14]:
fs = s3fs.S3FileSystem(profile=AWS_PROFILE)
qa = run_qa_checks(COUNTRY, SNAPSHOT_DATE, fs)
display(qa)

,feature_type,n_rows,n_files,geometry_types,centroid_valid_pct
0,PIPELINES,2040,27,"{'LineString': 1369, 'Point': 498, 'MultiPolyg...",100.0
1,POWER_LINES,9044,26,"{'LineString': 9041, 'MultiPolygon': 3}",100.0
2,POWER_SUBSTATIONS,2605,27,"{'MultiPolygon': 2598, 'Point': 7}",100.0
3,POWER_PLANTS,30449,27,"{'MultiPolygon': 27602, 'Point': 2847}",100.0
4,INDUSTRIAL_AREAS,4280,27,"{'MultiPolygon': 4148, 'Point': 126, 'LineStri...",100.0
5,STORAGE_TANKS,4145,27,"{'MultiPolygon': 3965, 'Point': 180}",100.0
6,FUEL_STATIONS,2634,27,"{'Point': 1359, 'MultiPolygon': 1275}",100.0
7,PORTS_TERMINALS,1282,26,"{'LineString': 1043, 'MultiPolygon': 141, 'Poi...",100.0
8,AIRPORTS,833,26,"{'LineString': 704, 'MultiPolygon': 89, 'Point...",100.0
9,ROADS,701721,27,"{'LineString': 532063, 'Point': 153745, 'Multi...",100.0


## Schema reference

| Column | Type | Description |
|--------|------|-------------|
| osm_id | string | OSM object ID (e.g. n123, w456, r789) |
| osm_type | string | node, way, relation |
| feature_type | string | PIPELINES, POWER_LINES, … |
| geometry | geometry | Point/LineString/Polygon (WGS84) |
| centroid_lon | double | Longitude of centroid/representative point |
| centroid_lat | double | Latitude of centroid/representative point |
| h3_6 | string | H3 cell ID at res 6 (~36 km²), separate column |
| h3_7 | string | H3 cell ID at res 7 (~5 km²), separate column |
| h3_8 | string | H3 cell ID at res 8 (~0.7 km²), separate column |
| h3_9 | string | H3 cell ID at res 9 (~0.1 km²), separate column |
| name | string | OSM name tag |
| tags_json | string | Pruned tags as JSON |
| snapshot_date | string | YYYY-MM-DD |
| country | string | ES |
| + promoted columns | string | Per feature type (pipeline, voltage, highway, …) |

## Schema verification (read-back test for Gold)

Checks if Parquet files can be read and if column types are consistent across partitions. Use to debug Athena `HIVE_BAD_DATA` errors.

## Low-level Parquet read (row-group by row-group)

When `pq.read_table` fails with "Unable to merge" (string vs dictionary), the data is still in the file – PyArrow just can't merge row groups. This cell reads each row group separately, decodes dictionary columns, and concatenates. Proves the data is readable.

In [15]:
# Read Parquet row-group by row-group (bypasses merge error), decode dict cols → use for Gold
def read_parquet_row_groups(path: str, s3_fs) -> pd.DataFrame:
    """Read single Parquet file; decode dictionary cols to avoid merge error. Full data, no Nones."""
    pf = pq.ParquetFile(s3_fs.open(path, "rb"))
    tables = []
    for i in range(pf.num_row_groups):
        rg = pf.read_row_group(i)
        for j in range(rg.num_columns):
            if pa.types.is_dictionary(rg.schema.field(j).type):
                rg = rg.set_column(j, rg.schema.field(j).name, rg.column(j).dictionary_decode())
        tables.append(rg)
    tbl = pa.concat_tables(tables)
    return tbl.to_pandas()

def read_silver_parquet_dir(prefix: str, s3_fs, feature_type: str | None = None) -> pd.DataFrame:
    """Read all Parquet files under prefix (optionally filtered by feature_type). Use for Gold."""
    base = f"{prefix}/feature_type="
    fts = [feature_type] if feature_type else FEATURE_TYPES
    dfs = []
    for ft in fts:
        p = f"{base}{ft}/"
        files = sorted(s3_fs.glob(f"{p}*.parquet"))
        for f in files:
            dfs.append(read_parquet_row_groups(f, s3_fs))
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# Single file
path = f"{S3_BUCKET}/{SILVER_PREFIX}/country={COUNTRY}/snapshot_date={SNAPSHOT_DATE}/feature_type=POWER_PLANTS/part-00002.parquet"
fs = s3fs.S3FileSystem(profile=AWS_PROFILE)
df = read_parquet_row_groups(path, fs)
print(f"Read OK: {len(df)} rows from part-00001.parquet")
print(df[["osm_id", "country", "plant_source", "generator_source"]].head())
print("\ngenerator_source value_counts:")
print(df["generator_source"].value_counts().head(10))
print("\ncountry:", df["country"].unique())
print("\nColumns:", list(df.columns))

# Example: read all POWER_PLANTS for Gold
base = f"{S3_BUCKET}/{SILVER_PREFIX}/country={COUNTRY}/snapshot_date={SNAPSHOT_DATE}"
all_pp = read_silver_parquet_dir(base, fs, feature_type="POWER_PLANTS")
# print(f"Gold-ready: {len(all_pp)} POWER_PLANTS")
all_pp['h3_6'].value_counts()

Read OK: 72 rows from part-00001.parquet
       osm_id country plant_source generator_source
0  w399679169      ES        solar             None
1  w399679170      ES        solar             None
2  w399679171      ES        solar             None
3  w399679172      ES        solar             None
4  w405183555      ES        solar             None

generator_source value_counts:
generator_source
solar     35
gas        3
diesel     1
Name: count, dtype: int64

country: ['ES']

Columns: ['osm_id', 'osm_type', 'feature_type', 'geometry', 'centroid_lon', 'centroid_lat', 'h3_6', 'h3_7', 'h3_8', 'h3_9', 'name', 'tags_json', 'snapshot_date', 'country', 'pipeline', 'substance', 'location', 'operator', 'ref', 'power', 'voltage', 'cables', 'substation', 'plant_source', 'generator_source', 'capacity', 'landuse', 'industrial', 'man_made', 'content', 'amenity', 'brand', 'opening_hours', 'harbour', 'aeroway', 'iata', 'icao', 'highway', 'surface', 'bridge', 'tunnel', 'oneway', 'maxspeed', 'access

h3_6
86391c467ffffff    3085
863903db7ffffff    1669
86391ab1fffffff    1618
86390e647ffffff    1593
86391c65fffffff    1561
                   ... 
86391c027ffffff       1
863919127ffffff       1
8639180f7ffffff       1
8639181a7ffffff       1
86390aa8fffffff       1
Name: count, Length: 721, dtype: int64

In [27]:
# Schema verification: check if we can read silver Parquet and if column types are consistent
import pyarrow.parquet as pq

def verify_silver_schema(country: str, snapshot_date: str, s3_fs: s3fs.S3FileSystem, feature_type: str | None = None):
    """
    For each feature_type (or a single one), read Parquet schemas from S3.
    Reports: schema per file, type mismatches across files, and a test read.
    """
    base = f"{S3_BUCKET}/{SILVER_PREFIX}/country={country}/snapshot_date={snapshot_date}"
    fts = [feature_type] if feature_type else FEATURE_TYPES

    for ft in fts:
        prefix = f"{base}/feature_type={ft}/"
        files = sorted(s3_fs.glob(f"{prefix}*.parquet"))
        if not files:
            print(f"  {ft}: no files")
            continue

        schemas = {}
        for path in files:
            uri = f"s3://{path}"
            try:
                tbl = pq.read_table(uri, filesystem=s3_fs)
                schemas[path] = tbl.schema
            except Exception as e:
                print(f"  {ft} READ FAILED {path}: {e}")
                continue

        if not schemas:
            print(f"  {ft}: all reads failed, skipping schema check")
            continue

        # Compare schemas
        ref = list(schemas.values())[0]
        ref_str = {f.name: str(f.type) for f in ref}
        mismatches = []
        for path, schema in schemas.items():
            for f in schema:
                if ref_str.get(f.name) != str(f.type):
                    mismatches.append((path.split("/")[-1], f.name, ref_str.get(f.name), str(f.type)))

        if mismatches:
            print(f"  {ft}: SCHEMA MISMATCHES:")
            for fname, col, t1, t2 in mismatches[:10]:
                print(f"    {fname} col={col}: ref={t1} vs {t2}")
            if len(mismatches) > 10:
                print(f"    ... and {len(mismatches)-10} more")
        else:
            print(f"  {ft}: OK ({len(files)} files, {ref.num_fields} cols)")

        # Sample promoted columns (e.g. plant_source) – use first successfully read file
        first_ok = next((p for p in files if p in schemas), None)
        if first_ok:
            promoted = ["plant_source", "generator_source", "capacity", "building_levels"]
            tbl = pq.read_table(f"s3://{first_ok}", filesystem=s3_fs)
            sample_types = [f"{c}={tbl.schema.field(c).type}" for c in promoted if tbl.schema.get_field_index(c) >= 0]
            print(f"    Sample types: " + ", ".join(sample_types) if sample_types else "    (no promoted cols in schema)")

# Run verification (optionally limit to problematic feature_type)
fs = s3fs.S3FileSystem(profile=AWS_PROFILE)
verify_silver_schema(COUNTRY, SNAPSHOT_DATE, fs, feature_type="POWER_PLANTS")  # or None for all

# Full read test (simulates Gold layer – use gpd.read_parquet for GeoParquet)
base = f"s3://{S3_BUCKET}/{SILVER_PREFIX}/country={COUNTRY}/snapshot_date={SNAPSHOT_DATE}/feature_type=POWER_PLANTS/"
try:
    df = gpd.read_parquet(base, storage_options=STORAGE_OPTIONS)
    print(f"\nRead OK: {len(df)} rows")
    cols = [c for c in ["plant_source", "capacity", "generator_source"] if c in df.columns]
    print(df[cols].head(3) if cols else df.iloc[:, :5].head(3))
except Exception as e:
    print(f"\nRead FAILED (e.g. schema mismatch): {e}")

  POWER_PLANTS READ FAILED ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=POWER_PLANTS/part-00001.parquet: The provided token has expired.
  POWER_PLANTS READ FAILED ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=POWER_PLANTS/part-00002.parquet: The provided token has expired.
  POWER_PLANTS READ FAILED ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=POWER_PLANTS/part-00003.parquet: The provided token has expired.
  POWER_PLANTS: all reads failed, skipping schema check

Read FAILED (e.g. schema mismatch): ie-datalake/silver/osm/features_energy/country=ES/snapshot_date=2026-02-25/feature_type=POWER_PLANTS
